<a href="https://colab.research.google.com/github/guilherme-argentino/fiap-ia4devs-techchallenge-fase5/blob/main/Fase5_Tech_Challenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fase 5 Tech Challenge

In [1]:
!git clone https://github.com/guilherme-argentino/fiap-ia4devs-techchallenge-fase5.git conteudo

Cloning into 'conteudo'...
remote: Enumerating objects: 149, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 149 (delta 3), reused 0 (delta 0), pack-reused 137 (from 2)
Receiving objects: 100% (149/149), 93.64 MiB | 12.57 MiB/s, done.
Resolving deltas: 100% (18/18), done.
Updating files: 100% (140/140), done.


In [2]:
!apt-get install git-lfs
!git lfs install

%cd /content/conteudo

!git lfs pull

%cd /content

!pip install -r conteudo/requirements.txt

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.3).
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.
Git LFS initialized.
/content/conteudo
/content
Reason for being yanked: deprecated, use 4.8.0.76
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 644.8/644.8 kB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.7/61.7 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 19.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 92.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 91.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 91.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.

In [1]:
import os
import sys

import cv2
from ultralytics import YOLO

# Set YOLO to quiet mode
os.environ['YOLO_VERBOSE'] = 'False'

def detect_and_highlight_objects_in_photo(image_path):
    image_orig = cv2.imread(image_path)

    yolo_model = YOLO('yolov8n.pt')  # load an official model
    yolo_model = YOLO('./conteudo/runs/detect/Normal_Compressed/weights/best.pt')

    results = yolo_model(image_orig)

    for result in results:
        classes = result.names
        cls = result.boxes.cls
        conf = result.boxes.conf
        detections = result.boxes.xyxy

        for pos, detection in enumerate(detections):
            if conf[pos] >= 0.5:
                xmin, ymin, xmax, ymax = detection
                label = f"{classes[int(cls[pos])]} {conf[pos]:.2f}"
                color = (0, int(cls[pos]), 255)
                cv2.rectangle(image_orig, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 2)
                cv2.putText(image_orig, label, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    result_path = "./imgs/Test/teste.jpg"
    cv2.imwrite(result_path, image_orig)
    return result_path

def detect_objects_in_video(video_path):

    yolo_model = YOLO('./conteudo/runs/detect/Normal_Compressed/weights/best.pt')
    yolo_model.verbose = False
    video_capture = cv2.VideoCapture(video_path)

    # Flag to track if any objects have been detected
    object_detected = False

    total_frames = int(video_capture.get(cv2.CAP_PROP_FRAME_COUNT))  # Obter o total de frames

    frame_number = 0  # Inicializar o contador de frames

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break

        frame_number += 1  # Incrementar o contador de frames

        results = yolo_model(frame, verbose=False)

        for result in results:
          classes = result.names
          cls = result.boxes.cls
          conf = result.boxes.conf
          detections = result.boxes.xyxy
          if len(detections) > 0: # Check if any detections are made
            for pos, detection in enumerate(detections):
              print(f"Frame {frame_number}/{total_frames} - {classes[int(cls[pos])]} {conf[pos]:.2f}")
            object_detected = True
            break  # Exit the loop if an object is detected

#        if object_detected:
#          break  # Exit the outer loop if an object is detected

    video_capture.release()

    return object_detected # Return True if object_detected is True, otherwise False

def detect_and_highlight_objects_in_video(video_path):
    yolo_model = YOLO('yolov8n.pt')  # load an official model
    yolo_model = YOLO('./conteudo/runs/detect/Normal_Compressed/weights/best.pt')
    video_capture = cv2.VideoCapture(video_path)
    width = int(video_capture.get(3))
    height = int(video_capture.get(4))
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    result_video_path = "detected_objects_video1.avi"
    out = cv2.VideoWriter(result_video_path, fourcc, 20.0, (width, height))

    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        results = yolo_model(frame)

        for result in results:
            classes = result.names
            cls = result.boxes.cls
            conf = result.boxes.conf
            detections = result.boxes.xyxy

            for pos, detection in enumerate(detections):
                if conf[pos] >= 0.5:
                    xmin, ymin, xmax, ymax = detection
                    label = f"{classes[int(cls[pos])]} {conf[pos]:.2f}"
                    color = (0, int(cls[pos]), 255)
                    cv2.rectangle(frame, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 2)
                    cv2.putText(frame, label, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

        out.write(frame)
    video_capture.release()
    out.release()

    return result_video_path

def detect_objects_and_plot(path_orig):
    image_orig = cv2.imread(path_orig)

    yolo_model = YOLO('yolov8n.pt')  # load an official model
    yolo_model = YOLO('./conteudo/runs/detect/Normal_Compressed/weights/best.pt')

    results = yolo_model(image_orig)

    for result in results:
        classes = result.names
        cls = result.boxes.cls
        conf = result.boxes.conf
        detections = result.boxes.xyxy

        for pos, detection in enumerate(detections):
            if conf[pos] >= 0.5:
                xmin, ymin, xmax, ymax = detection
                label = f"{classes[int(cls[pos])]} {conf[pos]:.2f}"
                color = (0, int(cls[pos]), 255)
                cv2.rectangle(image_orig, (int(xmin), int(ymin)), (int(xmax), int(ymax)), color, 2)
                cv2.putText(image_orig, label, (int(xmin), int(ymin) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1, cv2.LINE_AA)

    cv2.imshow("Teste", image_orig)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [2]:
video_path = 'conteudo/video1.mp4'

result = detect_objects_in_video(video_path)

print("\n\n#####################################################################")
if result:
    print("\n\nObjeto cortante ou arma encontrado? Sim")
else:
    print("\n\nObjeto cortante ou arma encontrado? Não")
print("\n\n#####################################################################")


/usr/local/lib/python3.11/dist-packages/ultralytics/nn/tasks.py:567: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(file, map_location='cpu'), file  # load


Frame 38/1264 - guns 0.30
Frame 39/1264 - guns 0.40
Frame 41/1264 - guns 0.39
Frame 42/1264 - guns 0.40
Frame 43/1264 - guns 0.28
Frame 44/1264 - guns 0.33
Frame 45/1264 - guns 0.27
Frame 54/1264 - guns 0.33
Frame 60/1264 - guns 0.27
Frame 61/1264 - guns 0.39
Frame 62/1264 - guns 0.39
Frame 71/1264 - guns 0.41
Frame 72/1264 - guns 0.41
Frame 73/1264 - guns 0.26
Frame 75/1264 - guns 0.49
Frame 79/1264 - guns 0.29
Frame 90/1264 - knife 0.46
Frame 104/1264 - guns 0.37
Frame 117/1264 - knife 0.27
Frame 118/1264 - guns 0.43
Frame 118/1264 - knife 0.28
Frame 122/1264 - knife 0.48
Frame 125/1264 - knife 0.28
Frame 126/1264 - knife 0.28
Frame 130/1264 - knife 0.34
Frame 134/1264 - knife 0.31
Frame 136/1264 - knife 0.28
Frame 139/1264 - knife 0.38
Frame 141/1264 - knife 0.30
Frame 142/1264 - knife 0.30
Frame 143/1264 - knife 0.27
Frame 144/1264 - knife 0.34
Frame 148/1264 - knife 0.27
Frame 148/1264 - knife 0.27
Frame 159/1264 - knife 0.26
Frame 160/1264 - knife 0.47
Frame 166/1264 - knife 0.58